 # Creat default box
 Tạo ra 8732 default box

In [18]:
from lib_all import *
from config import get_config
cfg = get_config()


In [19]:
class DefBox():
    def __init__(self, cfg):
        self.img_size = cfg["input_size"]
        self.feature_maps = cfg["feature_maps"]
        self.min_size = cfg["min_size"]
        self.max_size = cfg["max_size"]
        self.aspect_ratios = cfg["aspect_ratios"]
        self.steps = cfg["steps"]

    def create_defbox(self):
        """
        :return: [8732, 4]
        """
        defbox_list = []

        for index, size_grid in enumerate(self.feature_maps):
            for i, j in itertools.product(range(size_grid), repeat=2):
                f_k = self.img_size / self.steps[index]
                cx = (j+0.5)/f_k
                cy = (i+0.5)/f_k

                # small square box
                scale = self.min_size[index]/self.img_size #first case: 30/300
                defbox_list += [cx, cy, scale, scale]

                # big square box
                scale_ = sqrt(scale*(self.max_size[index]/self.img_size))
                defbox_list += [cx, cy, scale_, scale_]

                for ar in self.aspect_ratios[index]:
                    defbox_list += [cx, cy, scale*sqrt(ar), scale/sqrt(ar)]
                    defbox_list += [cx, cy, scale/sqrt(ar), scale*sqrt(ar)]

        output = torch.Tensor(defbox_list).view(-1, 4)
        output.clamp_(max = 1, min=0)

        return output

# Test

In [20]:
defbox = DefBox(cfg)
dbox_list = defbox.create_defbox()
print(dbox_list.shape)
    # print(pd.DataFrame(dbox_list.numpy()))

torch.Size([8732, 4])
